In [2]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())

1.13.1+cu116
False


In [3]:
import pandas as pd

# file_path = "data/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
file_path = "data/Monday-WorkingHours.pcap_ISCX.csv"
df = pd.read_csv(file_path)
df.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,7/7/2017 3:30,109,1,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,7/7/2017 3:30,52,1,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,7/7/2017 3:30,34,1,1,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,7/7/2017 3:30,3,2,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [7]:
# extract benign traffic
df_benign = df[df[' Label'] == 'BENIGN']
print(f"Number of benign traffic: {len(df_benign)}")
print(f"Number of total traffic: {len(df)}")

Number of benign traffic: 97718
Number of total traffic: 225745


Simulate a realistic scene where packets arrive one by one

In [ ]:
def cicids_parser(df, idx):
    src_addr = df.iloc[idx][' Source IP']
    proto_code = df.iloc[idx][' Protocol']
    # TODO

parse mirai dataset

In [7]:

from utils import pcap2csv

file_name = "mirai.pcap"
pcap2csv(file_name)

ImportError: cannot import name 'pcap2csv' from 'utils' (/root/wtz/Project/utils.py)

In [1]:
import pandas as pd
import os
from config import whisper_config
from utils import PacketMetaData, weight_transform
import torch

MIN_INTERVAL_TIME = 1e-5
IS_TRAINING = True

file_path = os.path.join("data", "mirai.csv")
print("Load data from: ", file_path)
df = pd.read_csv(file_path)

print("Start aggregating data...")
mp = {}
for idx in range(len(df)): # simulate the real-time traffic
    src_addr = df.iloc[idx]['src_addr']
    if src_addr not in mp:
        mp[src_addr] = []
    mp[src_addr].append(PacketMetaData(src_addr, df.iloc[idx]['proto_code'], 
                        df.iloc[idx]['pkt_length'], df.iloc[idx]['timestamp']))

print("Number of unique src_addr: ", len(mp))


Load data from:  data/mirai.csv
Start aggregating data...
Number of unique src_addr:  22


In [5]:
df_group = df.groupby('src_addr')
print(f"Number of unique src_addr: {len(df_group)}")

Number of unique src_addr: 22


In [3]:
print("Start transforming data...")
train_data = []
test_data = {}
for k, v in mp.items():
    if len(v) < whisper_config["n_fft"]:
        continue
    # calculate time interval
    for i in range(len(v) - 1, 0, -1):
        v[i].timestamp -= v[i - 1].timestamp
        if v[i].timestamp <= 0:
            v[i].timestamp = MIN_INTERVAL_TIME
    v[0].timestamp = MIN_INTERVAL_TIME

    # packet encoding
    ten = torch.tensor([weight_transform(x) for x in v])
    ten_fft = torch.stft(ten, whisper_config["n_fft"])
    ten_power = torch.pow(ten_fft[:, :, 0], 2) + torch.pow(ten_fft[:, :, 1], 2)
    ten_res = ((ten_power.squeeze()+1).log2()).permute(1, 0)
    ten_res = torch.where(torch.isnan(ten_res), torch.zeros_like(ten_res), ten_res)
    ten_res = torch.where(torch.isinf(ten_res), torch.zeros_like(ten_res), ten_res)

    # train model
    if (IS_TRAINING and len(train_data) < whisper_config["num_train_data"]):
        if (ten_res.size(0) > whisper_config["mean_win_train"]):
            for _ in range(whisper_config["num_train_sample"]):
                start_idx = torch.randint(0, ten_res.size(0) -1 - whisper_config["mean_win_train"], (1,)).item()
                ten_tmp = ten_res[start_idx:start_idx+whisper_config["mean_win_train"], :].mean(dim=0)
                train_data.append(ten_tmp.tolist())
        else:
            train_data.append(ten_res.mean(dim=0).tolist())
    else:
        tmp_data = []
        if (ten_res.size(0) > whisper_config["mean_win_test"]):
            for idx in range(0, ten_res.size(0) - whisper_config["mean_win_test"], whisper_config["mean_win_test"]):
                ten_tmp = ten_res[idx:idx+whisper_config["mean_win_test"], :].mean(dim=0)
                tmp_data.append(ten_tmp.tolist())
        else:
            tmp_data.append(ten_res.mean(dim=0).tolist())
        test_data[k] = tmp_data

if (len(train_data) < whisper_config["num_train_data"]):
    print(f"Number of training data {len(train_data)} is less than {whisper_config['num_train_data']}")
    exit(1)

Start transforming data...


/root/anaconda3/envs/PRO/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


In [4]:
if (IS_TRAINING):
    print("Start training...")
    train_data = torch.tensor(train_data)
    print("Training data shape: ", train_data.shape)
    # implement kmeans with train_data and val_K
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=whisper_config["val_K"], random_state=0).fit(train_data.cpu().numpy())
    # save centroids
    if (whisper_config["save_to_file"]):
        import json
        save_path = os.path.join(whisper_config["save_dir"], whisper_config["save_file_prefix"] + "centroids.json")
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        with open(save_path, "w") as f:
            json.dump(kmeans.cluster_centers_.tolist(), f)

Start training...
Training data shape:  torch.Size([501, 26])


/root/anaconda3/envs/PRO/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:

print("Start testing...")
import json
save_path = os.path.join(whisper_config["save_dir"], whisper_config["save_file_prefix"] + "centroids.json")
with open(save_path, "r") as f:
    centroids = json.load(f)
    centroids = torch.tensor(centroids)

print(f"centroids shape: {centroids.shape}")

test_res = []
for k, v in test_data.items():
    min_dist = 1e12
    v = torch.tensor(v)
    if (v.size(0) > whisper_config["mean_win_test"]):
        max_dist = 0
        for idx in range(0, v.size(0) - whisper_config["mean_win_test"], whisper_config["mean_win_test"]):
            ten_tmp = v[idx:idx+whisper_config["mean_win_test"], :].mean(dim=0)
            dist = torch.norm(ten_tmp - centroids, dim=1).min()
            max_dist = max(max_dist, dist) # find the max minimum distance among all segments
        min_dist = max_dist
    else:
        dist = torch.norm(v.mean(dim=0) - centroids, dim=1).min()
        min_dist = min(min_dist, dist)

    test_res.append({"src_addr": k, "loss": min_dist.item()})

res_path = os.path.join(whisper_config["save_dir"], whisper_config["save_file_prefix"] + "test_res.json")
os.makedirs(os.path.dirname(res_path), exist_ok=True)
with open(res_path, "w") as f:
    json.dump(test_res, f)


Start testing...
centroids shape: torch.Size([10, 26])


In [1]:
import torch

# for i in range(10):
#     print(torch.randint(0, 1, (1,)).item())

test_arr = [i for i in range(30)]
test_ten = torch.tensor(test_arr).float()
test_ten_fft = torch.stft(test_ten, 50)
print(test_ten_fft.shape)

test_ten_power = torch.pow(test_ten_fft[:, :, 0], 2) + torch.pow(test_ten_fft[:, :, 1], 2)
test_ten_res = ((test_ten_power+1).log2()).permute(1, 0)
print(test_ten_res.shape)

test_ten_res = torch.where(torch.isnan(test_ten_res), torch.zeros_like(test_ten_res), test_ten_res)
test_ten_res = torch.where(torch.isinf(test_ten_res), torch.zeros_like(test_ten_res), test_ten_res)

# test_ten_temp = test_ten_res[0:10, :].mean(dim=0)
test_ten_temp = test_ten_res.mean(dim=0)
print(test_ten_temp.shape)

# test_ten_power2 = torch.pow(test_ten_fft.permute(2, 0, 1)[0], 2) \
#                     + torch.pow(test_ten_fft.permute(2, 0, 1)[1], 2)
# print(test_ten_power2)
# test_ten_power2 = test_ten_power2.squeeze()
# print(test_ten_power2)
# test_ten_res2 = ((test_ten_power2+1).log2()).permute(1, 0)
# print(test_ten_res2)

torch.Size([6, 16, 2])
torch.Size([16, 6])
torch.Size([6])


/root/anaconda3/envs/PRO/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


In [9]:
import torch

arr = [i for i in range(100)]
n_fft = 50
ten = torch.tensor(arr).float()
ten_fft = torch.stft(ten, n_fft)
print(ten_fft.shape)

torch.Size([26, 9, 2])
